In [ ]:
import warnings; warnings.filterwarnings("ignore", message="Initializing zero-element tensors is a no-op")
import pandas as pd
import torch
import os 
from utils.eval_utils import get_pairs, eval_epochs, LFWSingleDataset

from utils.utils import transform
from models.NN2 import FaceNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

IMAGES_PATH = './data/lfw-faces/'
DF_PATH = './data/'
EPOCHS_PATH = './checkpoints/'

In [ ]:
test_df = pd.read_csv(os.path.join(DF_PATH, 'lfw_faces_test.csv'))
print(f'Test data shape: {test_df.shape}')

pairs = get_pairs(test_df, 1024)
print(f'Pairs shape: {pairs.shape}')

# **VAL, FAR, ROC, Dists**

In [ ]:
eval_epochs(epochs_path=EPOCHS_PATH, 
            pairs=pairs,
            model_class=FaceNet, 
            batch_size=32,
            transform=transform, 
            device=device,
            target_far=1e-3)

# **Visualizar os embeddings**

In [ ]:
import os
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def view_embs(epochs_path, model_class, test_df, transform, device):
    models_name = os.listdir(epochs_path)
    models_name = [model for model in models_name if 'bons' not in model]
    models_name_sorted = sorted(models_name, key=lambda x: int(x.replace('.pt', '').split('_')[1]))
    model = models_name_sorted[-1]
    
    modelo = model_class(emb_size=64).to(device)
    modelo.eval()

    test_dataset = LFWSingleDataset(test_df, transform=transform)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=0)

    model_path = os.path.join(epochs_path, model)
    load = torch.load(model_path)
    load = {k.replace('_orig_mod.', ''): v for k, v in load.items()}
    modelo.load_state_dict(load)
    
    embeddings = []
    with torch.no_grad():
        for (imgs, labels) in test_dataloader:
            images = imgs.to(device)
            embeddings.append(modelo(images).cpu().numpy())
    embeddings = np.concatenate(embeddings)
    
    # Reduzir dimensionalidade com t-SNE
    tsne = TSNE(n_components=2, perplexity=30, max_iter=300)
    embeddings_tsne = tsne.fit_transform(embeddings)
    
    # Reduzir dimensionalidade com PCA
    pca = PCA(n_components=2)
    embeddings_pca = pca.fit_transform(embeddings)
    
    plt.figure(figsize=(24, 8))
    
    # Plotar t-SNE
    plt.subplot(1, 2, 1)
    scatter = plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], c=test_df['id'], cmap='tab10', s=5)
    plt.colorbar(scatter)
    plt.title(f't-SNE {model}')
    
    # Plotar PCA
    plt.subplot(1, 2, 2)
    scatter = plt.scatter(embeddings_pca[:, 0], embeddings_pca[:, 1], c=test_df['id'], cmap='tab10', s=5)
    plt.colorbar(scatter)
    plt.title(f'PCA {model}')
    
    plt.show()

view_embs(EPOCHS_PATH, FaceNet, test_df, transform, device)